In [ ]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import cudf
import optuna
import importlib

sys.path.append(os.path.abspath(".."))

import src.models.rfc.rfc_cv_trainer as cv
import src.models.rfc.rfc_optuna_optimizer as op
import src.utils.telegram as te

In [ ]:
# load data
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

tr_df1 = cudf.read_parquet("../artifacts/features/base/tr_df1.parquet").astype("float32")

In [ ]:
# Tuning
importlib.reload(cv)
importlib.reload(op)
objective = op.create_objective(tr_df1)

op.run_optuna_search(
    objective,
    n_trials=50,
    direction="minimize",
    study_name="rfc_v1",
    storage=url,
    sampler=optuna.samplers.TPESampler(
        n_startup_trials=20, seed=42)
)
te.send_telegram_message("RFC Training Complete!")